In [1]:
from pyspark import SparkContext, SparkConf
import pandas as pd
import re
from operator import add
import numpy as np

In [2]:
sc = SparkContext.getOrCreate()
questionFile=sc.textFile("Questions.csv")
answerFile=sc.textFile("Answers.csv")
tagFile = sc.textFile("Tags.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [ ]:
#Read sample of question.csv(5%), generate the keywords list to focus on

In [3]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.count())

1264216


In [4]:
questionsample=question.sample(False,0.05,81)
questionLower = questionsample.map(lambda x: (x[0], (x[5]+x[6]).lower()))
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.count())

13348045


In [5]:
stopWordsList = stopWordsFile.collect()
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
print(N)

63340


In [6]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.count())

63340


In [7]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.count())
questionPairStage4.take(5)

63340


[('292860',
  ['esb=0.31955710438882234',
   'integration=0.26415616399604536',
   'jdbc=0.26063689583132227',
   'basis=0.2536044558888481',
   'redeploy=0.2316600941696921',
   'module=0.14232311356590852',
   'highly=0.13782171770001034']),
 ('432500',
  ['helptext=0.31347145520530423',
   'feature=0.3039788295804193',
   'sql=0.2814420402886694',
   'procedures=0.21630610982917833',
   'enterprise=0.2050126542510726',
   'studio=0.13212127154225234',
   'g=0']),
 ('435980',
  ['recompile=0.4738906693757042',
   'justifiable=0.3484750543457793',
   'declared=0.26399868358459483',
   'health=0.22104489800628596',
   'commented=0.1848018136332991',
   'find=0.1141603024729195',
   'sql=0.09390712143738686']),
 ('525430',
  ['gameconfig=0.39542916360950864',
   'getscreensize=0.21499596093759327',
   'getextendedstate=0.15153806178919574',
   'graphicsconfiguration=0.15153806178919574',
   'screenmaximized=0.15153806178919574',
   'windowy=0.14203773679396084',
   'windowx=0.1420377367

In [8]:
def fmap1(x):
    output=""
    for i in range(num_keywords):
        y=x[1][i].split("=")
        if float(y[1])>0.2:
           output+=y[0]+";"
    return ("key",output)
keywordlist=questionPairStage4.map(fmap1).reduceByKey(add)
keywords=keywordlist.take(1)
words=keywords[0][1][:-1]
keywordset=set(words.split(";"))
print(len(keywordset))
print(keywordset)

89383
{'differnce', 'rawargs', 'ltsp', 'newxpath', 'jsonserializer', 'selectedsupervisorname', 'myactionbartabs', 'portfolioitems', 'vdpau', 'review', 'leuprm', 'kostas', 'thepath', 'kljb', 'pathauto', 'standardised', 'submoudle', 'thecount', 'preceding', 'passe', 'gu', 'crop', 'mainlinks', 'objectxy', 'zqhh', 'pcfreq', 'avengers', 'trimmingtextblock', 'recibir', 'crawled', 'enablecors', 'txtemail', 'queuelist', 'gvid', 'chang', 'binsearch', 'onplayerregister', 'irregularly', 'bko', 'matthy', 'updateimpression', 'pikaday', 'fbsdkloginbehaviornative', 'mistakes', 'dummydf', 'effectiveuser', 'specialisation', 'leftmousebuttonup', 'ffffcc', 'nsdate', 'rbind', 'getlookandfeeldefaults', 'parnew', 'boomr', 'autodate', 'toolpointdragger', 'previousrange', 'ddhash', 'myhandler', 'compiledwrapper', 'api', 'justskins', 'addd', 'testtmpblock', 'tvprovidersuburb', 'heard', 'msofiledialogopen', 'lastdiskcheck', 'usesubmitbeheviour', 'eminem', 'dismissmodalviewcontrolleranimated', 'program', 'checks

In [ ]:
#Read tags -- too slow, omit

In [4]:
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
def f0(x): return re.sub('[^a-zA-Z]+', ';', x)
tagLower = tagLower.mapValues(f0)

def fmap1(x):
    return ("key",x[1]+";")
taglist=tagLower.map(fmap1).reduceByKey(add)
print(taglist.take(5))
#tagwords=taglist.take(1)
#words=keywords[0][1][:-1]
#tagset=set(words.split(";"))
#print(len(tagset))
#print(tagset)

KeyboardInterrupt: 

In [ ]:
#turn oringinal question pairs to pairs only keep the keywords,get 7 keywords for each question

In [ ]:
questionLower = question.map(lambda x: (x[0], (x[5]+x[6]).lower()))
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
questionPair=questionPairRaw.filter(lambda x: x[1] in keywordset)
print(questionPair.count())

In [ ]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
N = len(questionPair.countByKey())
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
stage2Map = questionPairStage2MapPhase.countByKey()
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.count())
print(questionPairStage2.take(5))

In [ ]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

In [ ]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))

num_keywords = 7
def f4_1(a,b):
    c=[]
    for i in range(num_keywords):
        if float(b.split("=")[1])>float(a[i].split("=")[1]):
            a[i]=b
            break
    return a
def f4_2(a,b):
    for i in range(num_keywords): #b
        for j in range(num_keywords): #a
            if float(b[i].split("=")[1])>float(a[j].split("=")[1]):
                a[j] = b[i]
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0","f=0","g=0"],f4_1,f4_2)
print(questionPairStage4.take(5))

In [ ]:
outMap=questionPairStage4.collectAsMap()
tagWithHeader = tagFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = tagWithHeader.first() #extract header
tag = tagWithHeader.filter(lambda x: x != header)
tagLower = tag.map(lambda x: (x[0], x[1].lower()))
def f0(x): return re.sub('[^a-zA-Z]+', '', x)
tagLower = tagLower.mapValues(f0)
tag_1000 = tagLower.map(lambda x: (x[0], x[1]+"=1000"))
questionPairStage5=questionPairStage4.map(lambda x:(x[0],x[1][0]+":"+x[1][1]+":"+x[1][2]+":"+x[1][3]+":"+x[1][4]+":"+x[1][5]+":"+x[1][6]) )
print(questionPairStage5.take(5))

In [ ]:
questionPairStage6=questionPairStage5.union(tag_1000)
def f6_1(a,b):
    return a+":"+b
def f6_2(a,b):
    return a+b
questionPairStage6=questionPairStage6.aggregateByKey("",f6_1,f6_2).map(lambda x: (x[0], x[1][1:]))
def f7(x):
    listx= x[1].split(":")
    listx.sort(key=lambda item:float(item.split("=")[1]),reverse=True)
    output=""
    for i in range(num_keywords):
        output+=":"+listx[i]
    return (x[0],output)
questionPairStage7=questionPairStage6.map(f7).map(lambda x: (x[0], x[1][1:]))
print(questionPairStage7.take(2))